In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.externals import joblib
%matplotlib inline

In [3]:
import keras

Using TensorFlow backend.


In [4]:
def norm(arr, mins, maxs):
    arr = arr.astype(float)
    for i in range(arr.shape[-1]):
        arr[:, :, i] -= mins[i]
        arr[:, :, i] /= maxs[i]
    return arr

In [5]:
mins_audio, maxs_audio = np.load('data/audio_mins.npy'), np.load('data/audio_maxs.npy')
mins_kinect, maxs_kinect = np.load('data/kinect_mins.npy'), np.load('data/kinect_maxs.npy')

## Подготовим модель

In [5]:
X_audio = np.concatenate([norm(np.load('data/X_train_audio.npy'), mins_audio, maxs_audio), 
                          norm(np.load('data/X_val_audio.npy'), mins_audio, maxs_audio)])
X_eyes = np.concatenate([np.load('data/X_train_eyes.npy'), np.load('data/X_val_eyes.npy')])
X_face = np.concatenate([np.load('data/X_train_face.npy'), np.load('data/X_val_face.npy')])
X_kinect = np.concatenate([norm(np.load('data/X_train_kinect.npy'), mins_kinect, maxs_kinect), 
                           norm(np.load('data/X_val_kinect.npy'),mins_kinect, maxs_kinect)])

print X_audio.shape, X_eyes.shape, X_face.shape, X_kinect.shape

(10446, 400, 36) (10446, 200, 6) (10446, 200, 100) (10446, 60, 27)


In [6]:
y = np.concatenate([np.load('data/y_train.npy'), np.load('data/y_val.npy')])

print y.shape

(10446,)


In [7]:
model = keras.models.load_model('models/model_e2e_phase2.hf5')
model = keras.models.Model(model.inputs, model.get_layer('average_1').output)

In [8]:
nn = model.predict([X_audio, X_eyes, X_face, X_kinect])

In [9]:
predictor = RandomForestClassifier(n_estimators=1000, n_jobs=4).fit(nn, y)

In [ ]:
joblib.dump(predictor, 'models/predictor_phase2_e2e.pkl')